<a href="https://colab.research.google.com/github/HILALOZTEMEL/Multi_Camera_Motion/blob/main/Multi_cam_motion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import the necessary packages
from __future__ import print_function

from imutils.video import VideoStream
import numpy as np
import time
import cv2


# Define range of pink color in HSV
lower_pink = np.array([140, 50, 50])  # Lower bound of pink hue
upper_pink = np.array([170, 255, 255])  # Upper bound of pink hue



# initialize the video streams and allow them to warmup
print("[INFO] starting cameras...")
webcam = VideoStream(src=0).start()
webcam1 = VideoStream(src=1).start()
time.sleep(2.0)



# loop over frames from the video streams
while True:
    # initialize the list of frames that have been processed
    frames = []
    # loop over the frames and their respective motion detectors
    for (stream) in (webcam, webcam1):
        # read the next frame from the video stream and resize
        # it to have a maximum width of 400 pixels
        frame = stream.read()
        frame = cv2.resize(frame,(480,480))


        hsv=cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
        gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        gray_blurred = cv2.medianBlur(gray,5)
        circles=cv2.HoughCircles(gray_blurred,cv2.HOUGH_GRADIENT,1,frame.shape[0]/4,param1=200,param2=45,minRadius=1,maxRadius=1200)
        #4535 max radius 1200 lük çember için
        answer=[0,0,0]
        found=""
        if circles is not None:

                circles=np.uint16(np.around(circles))

                for x,y,r in circles[0,:]:
                    width = 480  # Replace with the actual width of your image
                    height = 480

                    if 0 <= x < width and 0 <= y < height:
                        if np.all(np.greater(hsv[y][x], lower_pink)) and np.all(np.less_equal(hsv[y][x], upper_pink)):


                            cv2.circle(frame,(x,y),r,(0,255,0),2)

                            found+='pink'

                            answer=[x,y,2]

                        else:

                            answer=[0,0,0]

                            found+='no'



        # update the frames list
        frames.append(frame)

    # loop over the frames a second time
    for (frame, name) in zip(frames, ("Webcam", "Webcam1")):


        cv2.imshow(name, frame)
    # check to see if a key was pressed
    key = cv2.waitKey(1) & 0xFF
    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break
# do a bit of cleanup
print("[INFO] cleaning up...")
cv2.destroyAllWindows()
webcam.stop()
webcam1.stop()